In [52]:
import pandas as pd
import numpy as np

In [53]:
test = pd.read_csv('../data/test.csv', index_col=0)
weather = pd.read_csv('../data/weather_cleaned.csv')
spray = pd.read_csv('../data/spray_cleaned.csv')

In [54]:
test.columns = test.columns.map(lambda x: x.lower())
weather.columns = weather.columns.map(lambda x: x.lower())

In [55]:
test.species = test.species.map({'CULEX PIPIENS/RESTUANS': 'CULEX PIPIENS/RESTUANS',
                   'CULEX RESTUANS': 'CULEX RESTUANS',
                   'CULEX PIPIENS': 'CULEX PIPIENS',
                   'CULEX TERRITANS': 'CULEX OTHER', 
                   'CULEX SALINARIUS': 'CULEX OTHER',
                   'CULEX TARSALIS': 'CULEX OTHER',
                   'CULEX ERRATICUS': 'CULEX OTHER'})

test.species = test.species.fillna('CULEX OTHER')

In [56]:
test.columns

Index(['date', 'address', 'species', 'block', 'street', 'trap',
       'addressnumberandstreet', 'latitude', 'longitude', 'addressaccuracy'],
      dtype='object')

In [57]:
test['station'] = np.where(test['latitude'] >= 41.892, 1, 2)

In [58]:
test_weather = pd.merge(test, weather, on=['date', 'station'], )

In [59]:
train = pd.read_csv('../data/train_weather_spray_merged.csv')

In [60]:
set(train.columns).difference(test_weather.columns)

{'nummosquitos', 'spray_nearby', 'wnvpresent'}

The only final feature to engineer for test data is the `spray_nearby` feature.

In [61]:
from math import sin, cos, radians, asin, sqrt
def global_distance(lon1, lat1, lon2, lat2):
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 3956
    
    return c * r

In [11]:
traps = {}
for index, row in test_weather.iterrows():
    traps[row['trap']] = (row['longitude'], row['latitude'])


In [12]:
trap_distances = {}

for trap in traps:
    lon, lat = traps[trap]
    
    for index, spray_row in spray.iterrows():
        tmp_dist = global_distance(lon, lat, spray_row['Longitude'], spray_row['Latitude'])
        if trap in trap_distances:
            trap_distances[trap] = min(tmp_dist, trap_distances[trap])
        else:
            trap_distances[trap] = tmp_dist

These are the only features we're actually using for our models, so we'll store our testing data so that it can be easily accessed by these names.


In [62]:
cols = ['latitude', 'longitude', 'addressaccuracy', 'spray_nearby', 'station',
       'tmax', 'tmin', 'tavg', 'dewpoint', 'wetbulb', 'heat', 'cool',
       'preciptotal', 'stnpressure', 'sealevel', 'resultspeed', 'resultdir',
       'avgspeed', 'ts', 'sq', 'fg+', 'gr', 'br', 'tsra', 'dz', 'bcfg', 'hz',
       'fu', 'sn', 'fg', 'vcts', 'ra', 'mifg', 'vcfg', 'species_CULEX OTHER',
       'species_CULEX PIPIENS', 'species_CULEX PIPIENS/RESTUANS',
       'species_CULEX RESTUANS']

In [63]:
test_weather['spray_nearby'] = (test_weather['trap'].map(trap_distances) < .125).map(float)

In [64]:
test_weather_dummies = pd.get_dummies(test_weather, columns=['species'])

Just doing a brief check that the shape is what we expect, we should have 38 columns.

In [65]:
test_weather_dummies = test_weather_dummies[cols]

In [66]:
test_weather_dummies.shape

(116293, 38)

We still have one object type, `preciptotal` and we should convert the 'T' for trace amounts to a very small floating point like we did in the training data.

In [67]:
test_weather_dummies.select_dtypes(include='O').columns

Index(['preciptotal'], dtype='object')

In [68]:
test_weather_dummies['preciptotal'] = test_weather_dummies['preciptotal'].map(lambda x: x.replace('T', '.001')).astype(float)

In [69]:
test_weather_dummies.to_csv('../data/test_merged.csv', index=False)